In [3]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.layers import Dropout
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


# Preprocess the images
def preprocess_image(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            corrupted_images.append(image_path)
            return None
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (128, 128))  # Xception input size
        return image
    except Exception:
        corrupted_images.append(image_path)
        return None

# Load the dataset
def load_artist_data_as_dataframe(data_dir):
    data = []  
    label_map = {}
    corrupted_images = []

    for idx, folder in enumerate(os.listdir(data_dir)):
        if folder.startswith("pins_"):
            artist_name = folder.split('_', 1)[1]
            folder_path = os.path.join(data_dir, folder)
            num_images = len([file for file in os.listdir(folder_path) if file.endswith(('.jpg', '.jpeg', '.png'))])
            label_map[artist_name] = (idx, num_images)

    # Sort the label_map by the number of images and select the biggest 55 classes
    sorted_label_map = dict(sorted(label_map.items(), key=lambda item: item[1][1], reverse=True)[:55])

    # Remap the labels to be in the range 0 to 54
    new_label_map = {artist_name: new_idx for new_idx, (artist_name, _) in enumerate(sorted_label_map.items())}

    for artist_name, (old_idx, _) in sorted_label_map.items():
        folder = f"pins_{artist_name}"
        folder_path = os.path.join(data_dir, folder)
        
        for file in os.listdir(folder_path):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(folder_path, file)
                image = preprocess_image(file_path)
                if image is not None:
                    data.append({
                        "image_array": image,
                        "image_path": file_path,
                        "label": new_label_map[artist_name],
                        "artist_name": artist_name
                    })
    
    df = pd.DataFrame(data)
    return df, new_label_map

#data_dir = "pics"
data_dir = "105_classes_pins_dataset"
df, label_map = load_artist_data_as_dataframe(data_dir)

# Train-test split
df = df.groupby('label').filter(lambda x: len(x) >= 2)
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

X_train = np.stack(train_df['image_array'].values)
y_train = train_df['label'].values

X_test = np.stack(test_df['image_array'].values)
y_test = test_df['label'].values

# Normalize for Xception
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the Xception base model (pre-trained on ImageNet)
base_model = Xception(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the base model's layers
for layer in base_model.layers:
    layer.trainable = False

# Custom classification layers on top of Xception
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(len(np.unique(y_train)), activation='softmax')(x)

# Combine base model and new layers
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Print the model summary
model.summary()

# Training with data augmentation
train_generator = datagen.flow(X_train, y_train, batch_size=32)
val_generator = datagen.flow(X_test, y_test, batch_size=32)

# Train the model
history = model.fit(
    train_generator,
    epochs=150,
    validation_data=val_generator
)

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 63, 63,    │        864 │ input_layer_1[0]… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 63, 63,    │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 63, 63,    │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 61, 61,    │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 61, 61,    │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 61, 61,    │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 61, 61,    │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 61, 61,    │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 61, 61,    │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 61, 61,    │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 61, 61,    │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 31, 31,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 31, 31,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 31, 31,    │        512 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_12 (Add)        │ (None, 31, 31,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 31, 31,    │          0 │ add_12[0][0]    

 Total params: 21,938,783 (83.69 MB)

 Trainable params: 1,077,303 (4.11 MB)

...

Epoch 1/150


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1734533647.136572      92 service.cc:145] XLA service 0x7a303c0038e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734533647.136663      92 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  3/267 ━━━━━━━━━━━━━━━━━━━━ 18s 69ms/step - accuracy: 0.0226 - loss: 4.2396 

I0000 00:00:1734533652.374599      92 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


267/267 ━━━━━━━━━━━━━━━━━━━━ 52s 160ms/step - accuracy: 0.0360 - loss: 4.0037 - val_accuracy: 0.0864 - val_loss: 3.5968
Epoch 2/150
267/267 ━━━━━━━━━━━━━━━━━━━━ 36s 131ms/step - accuracy: 0.0650 - loss: 3.6464 - val_accuracy: 0.1305 - val_loss: 3.4217
Epoch 3/150
267/267 ━━━━━━━━━━━━━━━━━━━━ 36s 131ms/step - accuracy: 0.0984 - loss: 3.4811 - val_accuracy: 0.1376 - val_loss: 3.3093
Epoch 4/150
267/267 ━━━━━━━━━━━━━━━━━━━━ 36s 132ms/step - accuracy: 0.1208 - loss: 3.3781 - val_accuracy: 0.1521 - val_loss: 3.2372
Epoch 5/150
267/267 ━━━━━━━━━━━━━━━━━━━━ 36s 132ms/step - accuracy: 0.1296 - loss: 3.3158 - val_accuracy: 0.1704 - val_loss: 3.1875
Epoch 6/150
267/267 ━━━━━━━━━━━━━━━━━━━━ 36s 132ms/step - accuracy: 0.1398 - loss: 3.2524 - val_accuracy: 0.1789 - val_loss: 3.1452
Epoch 7/150
267/267 ━━━━━━━━━━━━━━━━━━━━ 37s 135ms/step - accuracy: 0.1535 - loss: 3.1918 - val_accuracy: 0.1793 - val_loss: 3.0994
Epoch 8/150
267/267 ━━━━━━━━━━━━━━━━━━━━ 37s 134ms/step - accuracy: 0.1666 - loss: 3.135

In [4]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {test_loss}, Accuracy: {test_accuracy * 100}%")

67/67 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.3602 - loss: 2.3967
Loss: 2.3905229568481445, Accuracy: 34.69483554363251%


In [5]:
train_loss12, train_accuracy12 = model.evaluate(X_train, y_train)
print(f"Loss: {train_loss12}, Accuracy: {train_accuracy12 * 100}%")

267/267 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.5363 - loss: 1.6542
Loss: 1.66472327709198, Accuracy: 53.38028073310852%


In [7]:
model.save("xception_face_classification_best_72.keras")